In [1]:
import os
from astro import constants as c
import numpy as np
from minirats.HaloFinder.py.haloCatalog import haloCatalog as hC

## Create parameter file

In [43]:
parameter_file_name = 'params_fid.dat'
in_spectrum_repo         = False

################Parameters#########################
output_rascas = '/cral/mauerhofer/RascasFiles/'
simu = 'simu_jeremy'
timesteps = [162,163] #range(107,165)
ion  = 'CII'

#Rascas
spectrum_rep_name = 'spectra_108'
IC_photons_name   = 'IC_photons_1.ic'
output_name   = 'output_fid.out'
nbundle = 1000

#Mock
ndir = 108

#Gas
vturb = 20.

#Lines

#CDD
ramDir = '/cral4/simulations/P13-20h-1Mpc-MUSIC/Zoom-7-10508/SPHINX_run/'
zoom = True
max_cells = 10000000

#Dust
albedo = 0.34
g_dust = 0.72
dust_model = 'SMC'

#Photstars
# IC_repo = '%s/%s'%(spectrum_rep,IC_photons_name)
weights = '/home/cral/mauerhofer/seds/PowLaw_Tables/PowLawTable.dat'
lmin = 1329
lmax = 1340
nphot = 1000000
seed = 2345

#write_ion
write_repo = '/cral/mauerhofer/%s/maps/'%(simu)
file_name = 'cells'
radius_factor = 2.7

#spectra
sedDir = '/home/cral/mauerhofer/seds/bpass100'
CD_lmin = 1332
CD_lmax = 1336

#IC_CD
method_IC_CD = 'fromstars'
xphot = [10,10,10]
IC_repo_CD = '%s/%s/%05d/%s/CD/IC_CD'%(output_rascas,simu,i,ion)

#CD
ndir_CD = 4
vmin_CD = -500
vmax_CD = 500
nBins_CD = 40

for i in timesteps:
    repo = '%s/%s/%05d/%s'%(output_rascas,simu,i,ion)
    domdump_rep = '%s/%s/%05d/%s/domdump/'%(output_rascas,simu,i,ion)
    spectrum_rep = '%s/%s/%05d/%s/%s/'%(output_rascas,simu,i,ion,spectrum_rep_name)
    
    if not os.path.exists('%s/%s/%05d/%s/domdump'%(output_rascas,simu,i,ion)):
        os.makedirs('%s/%s/%05d/%s/domdump/'%(output_rascas,simu,i,ion))
    if not os.path.exists(spectrum_rep):
        os.makedirs(spectrum_rep)
    if not os.path.exists('%s/%05d/%s'%(write_repo,i,ion)):
        os.makedirs('%s/%05d/%s'%(write_repo,i,ion))

    rascas_params = '!-------------------------------------------------------------------------------- \n'
    rascas_params = rascas_params + '[RASCAS] \n'
    rascas_params = rascas_params + '  DomDumpDir   = %s \n'%domdump_rep
    rascas_params = rascas_params + '  DomDumpFile  = domain_decomposition_params.dat \n'
    rascas_params = rascas_params + '  PhotonICFile = %s/%s \n'%(spectrum_rep,IC_photons_name)
    rascas_params = rascas_params + '  fileout      = %s/%s \n'%(spectrum_rep,output_name)
    rascas_params = rascas_params + '  nbundle      = %i \n'%nbundle
    rascas_params = rascas_params + '  verbose      = T \n'
    rascas_params = rascas_params + '!-------------------------------------------------------------------------------- \n \n'
    
    mock_params = '!-------------------------------------------------------------------------------- \n'
    mock_params = mock_params + '[mock] \n'
    mock_params = mock_params + '  nDirections         = %i \n'%ndir
    mock_params = mock_params + '  mock_parameter_file = %s/params_mock.dat \n'%(spectrum_rep)
    mock_params = mock_params + '  mock_outputfilename = %s/%s \n'%(spectrum_rep,output_name)
    mock_params = mock_params + '!--------------------------------------------------------------------------------\n \n'
    
    gas_params = '!-------------------------------------------------------------------------------- \n'
    gas_params = gas_params + '[gas_composition] \n'
    gas_params = gas_params + '  f_ion          = 0.01 \n'
    gas_params = gas_params + '  Zref           = 0.005 \n'
    gas_params = gas_params + '  v_turb         = %.2f \n'%vturb
    gas_params = gas_params + '  no_dust        = F \n'
    gas_params = gas_params + '  no_v           = F \n'
    gas_params = gas_params + '  file_excited   = T \n'
    gas_params = gas_params + '!--------------------------------------------------------------------------------\n \n'

    line_params = get_line_params(ion)
    cdd_params = get_cdd_params(ramDir,i,domdump_rep,zoom,max_cells)
    
    dust_params = '!-------------------------------------------------------------------------------- \n'
    dust_params = dust_params + '[dust] \n'
    dust_params = dust_params + '  albedo     = %.2f \n'%albedo
    dust_params = dust_params + '  g_dust     = %.2f \n'%g_dust
    dust_params = dust_params + '  dust_model = %s \n'%dust_model
    dust_params = dust_params + '!--------------------------------------------------------------------------------\n \n'
    
    phot_star_params = get_phot_star_params(ramDir,i,'%s/%s'%(spectrum_rep,IC_photons_name),zoom,weights,lmin,lmax,nphot,seed)
    write_ions_params = get_write_ions_params(ramDir,i,'%s/%05d/%s/%s'%(write_repo,i,ion,file_name),radius_factor,'%s/params_ions.dat'%domdump_rep,zoom,max_cells)
    
    spectra_params  = '!-------------------------------------------------------------------------------- \n'
    spectra_params = spectra_params + '[spectra] \n'
    spectra_params = spectra_params + '  sed_dir = %s \n'%sedDir
    spectra_params = spectra_params + '  nSEDgroups = 1 \n'
    spectra_params = spectra_params + '  group_decomp = %.2f, %.2f \n'%(c.hc_AeV/CD_lmax, c.hc_AeV/CD_lmin)
    spectra_params = spectra_params + '!--------------------------------------------------------------------------------\n \n'
    
    IC_CD_params = get_IC_CD_params(ramDir,i,'%s/%s/%05d/%s/CD/IC_CD'%(output_rascas,simu,i,ion),zoom,method_IC_CD,xphot)
    
    CD_params = '!-------------------------------------------------------------------------------- \n'
    CD_params = CD_params + '[ColumnDensity] \n'
    CD_params = CD_params + '  DomDumpDir     = %s \n'%domdump_rep
    CD_params = CD_params + '  RaysICFile     = %s/%s/%05d/%s/CD/IC_CD \n'%(output_rascas,simu,i,ion)
    CD_params = CD_params + '  fileout        = %s/%s/%05d/%s/CD/CD_out.out \n'%(output_rascas,simu,i,ion)
    CD_params = CD_params + '  nDirections    = %i \n'%ndir_CD
    CD_params = CD_params + '  direction_file = %s/%s/%05d/%s/spectra_4/params_mock.dat \n'%(output_rascas,simu,i,ion)
    CD_params = CD_params + '  vmin           = %.2f \n'%vmin_CD
    CD_params = CD_params + '  vmax           = %.2f \n'%vmax_CD
    CD_params = CD_params + '  nBins          = %i \n'%nBins_CD
    CD_params = CD_params + '!-------------------------------------------------------------------------------- \n \n'
    
    ramses_params = '!-------------------------------------------------------------------------------- \n'
    ramses_params = ramses_params + '[ramses] \n'
    ramses_params = ramses_params + '  star_reading_method = default \n  self_shielding      = F \n  ramses_rt           = T \n' \
        + '  verbose             = T \n  cosmo               = T \n  use_initial_mass    = T \n  use_proper_time     = T \n'
    ramses_params = ramses_params + '!--------------------------------------------------------------------------------\n \n'
    
    master_params = '!-------------------------------------------------------------------------------- \n'
    master_params = master_params + '[master] \n  verbose = T \n'
    master_params = master_params + '!--------------------------------------------------------------------------------\n \n'
    
    params_tot = rascas_params + mock_params + gas_params + line_params + cdd_params + dust_params + phot_star_params \
    + write_ions_params + spectra_params + IC_CD_params + CD_params + ramses_params + master_params
    
    if(in_spectrum_repo):
        f = open('%s/%s/%s'%(repo,spectrum_rep_name,parameter_file_name),'w')
    else:
        f = open('%s/%s'%(repo,parameter_file_name),'w')
    f.write(params_tot)
    f.close()
    

In [23]:
def get_IC_CD_params(ramDir,ts,IC_repo,zoom,method,xphot):
    IC_CD_params = '!-------------------------------------------------------------------------------- \n'
    IC_CD_params = IC_CD_params + '[IC_ColumnDensity] \n'
    IC_CD_params = IC_CD_params + '# input / output parameters: \n'
    IC_CD_params = IC_CD_params + '  repository           = %s \n'%ramDir
    IC_CD_params = IC_CD_params + '  snapnum              = %i \n'%ts
    IC_CD_params = IC_CD_params + '  outputfile           = %s \n \n'%IC_repo
    IC_CD_params = IC_CD_params + '# computational domain parameters \n'
    IC_CD_params = IC_CD_params + '  star_dom_type        =  sphere \n'
    
    if(not zoom):
        print('Have to implement for the case when its not a zoom')
        return 0
    hcat = hC(ramDir,ts,HaloDir='Halos/',zoom=zoom)
    readFromRamsesOutput = True
    hcat.load_catalog()
    hmain = np.where((hcat.level==1) &(hcat.contam < 0.5) &(hcat.mvir > 4e7*1e-11))[0]
    myhalo = hmain[np.where(hcat.mvir[hmain]==max(hcat.mvir[hmain]))[0][0]]
    center = [hcat.x_cu[myhalo],hcat.y_cu[myhalo],hcat.z_cu[myhalo]]
    radius = hcat.rvir_cu[myhalo]
    
    IC_CD_params = IC_CD_params + '  star_dom_pos         =  %.8f, %.8f, %.8f \n'%(center[0],center[1],center[2])
    IC_CD_params = IC_CD_params + '  star_dom_rsp         =  %.5f \n \n'%(radius*0.99)
    if(xphot[0]<1):
        IC_CD_params = IC_CD_params + '  x_phot               =  %.8f, %.8f, %.8f \n \n'%(xphot[0],xphot[1],xphot[2])
    else:
        IC_CD_params = IC_CD_params + '  x_phot               =  %.8f, %.8f, %.8f \n'%(center[0],center[1],center[2])
    IC_CD_params = IC_CD_params + '# miscelaneous : \n'
    IC_CD_params = IC_CD_params + '  verbose              = T \n'
    IC_CD_params = IC_CD_params + '  method               = %s \n'%method
    IC_CD_params = IC_CD_params + '!-------------------------------------------------------------------------------- \n \n'
    return IC_CD_params

In [24]:
def get_write_ions_params(ramDir,ts,write_repo,radius_factor,ion_file,zoom,max_cells):
    write_ions_params = '!-------------------------------------------------------------------------------- \n'
    write_ions_params = write_ions_params + '[write_ions] \n'
    write_ions_params = write_ions_params + '  write_path         = %s \n'%write_repo
    write_ions_params = write_ions_params + '  repository         = %s \n'%ramDir
    write_ions_params = write_ions_params + '  snapnum            = %i \n'%ts
    write_ions_params = write_ions_params + '  ion_parameter_file = %s \n'%ion_file
    
    if(not zoom):
        print('Have to implement for the case when its not a zoom')
        return 0
    hcat = hC(ramDir,ts,HaloDir='Halos/',zoom=zoom)
    readFromRamsesOutput = True
    hcat.load_catalog()
    hmain = np.where((hcat.level==1) &(hcat.contam < 0.5) &(hcat.mvir > 4e7*1e-11))[0]
    myhalo = hmain[np.where(hcat.mvir[hmain]==max(hcat.mvir[hmain]))[0][0]]
    center = [hcat.x_cu[myhalo],hcat.y_cu[myhalo],hcat.z_cu[myhalo]]
    radius = hcat.rvir_cu[myhalo]
    
    write_ions_params = write_ions_params + '  comput_dom_pos     =  %.8f, %.8f, %.8f \n'%(center[0],center[1],center[2])
    write_ions_params = write_ions_params + '  comput_dom_rsp     =  %.5f \n'%(radius*radius_factor)
    write_ions_params = write_ions_params + '  verbose            = T \n'
    write_ions_params = write_ions_params + '  max_cells          = %i \n'%max_cells
    write_ions_params = write_ions_params + '!--------------------------------------------------------------------------------\n \n'
    return write_ions_params



In [25]:
def get_phot_star_params(ramDir,ts,IC_repo,zoom,weights,lmin,lmax,nphot,seed):
    phot_star_params = '!-------------------------------------------------------------------------------- \n'
    phot_star_params = phot_star_params + '[PhotonsFromStars] \n'
    phot_star_params = phot_star_params + '# input / output parameters: \n'
    phot_star_params = phot_star_params + '  repository           = %s \n'%ramDir
    phot_star_params = phot_star_params + '  snapnum              = %i \n'%ts
    phot_star_params = phot_star_params + '  outputfile           = %s \n \n'%IC_repo
    phot_star_params = phot_star_params + '# computational domain parameters \n'
    phot_star_params = phot_star_params + '  star_dom_type        =  sphere \n'
    
    if(not zoom):
        print('Have to implement for the case when its not a zoom')
        return 0
    hcat = hC(ramDir,ts,HaloDir='Halos/',zoom=zoom)
    readFromRamsesOutput = True
    hcat.load_catalog()
    hmain = np.where((hcat.level==1) &(hcat.contam < 0.5) &(hcat.mvir > 4e7*1e-11))[0]
    myhalo = hmain[np.where(hcat.mvir[hmain]==max(hcat.mvir[hmain]))[0][0]]
    center = [hcat.x_cu[myhalo],hcat.y_cu[myhalo],hcat.z_cu[myhalo]]
    radius = hcat.rvir_cu[myhalo]
    
    phot_star_params = phot_star_params + '  star_dom_pos         =  %.8f, %.8f, %.8f \n'%(center[0],center[1],center[2])
    phot_star_params = phot_star_params + '  star_dom_rsp         =  %.5f \n \n'%(radius*0.99)
    phot_star_params = phot_star_params + '# Particle weights \n'
    phot_star_params = phot_star_params + '  weight_type          = PowLaw \n'
    phot_star_params = phot_star_params + '  weight_input_file    = %s \n'%weights
    phot_star_params = phot_star_params + '# Spectral shape \n'
    phot_star_params = phot_star_params + '  spec_type            = PowLaw \n'
    phot_star_params = phot_star_params + '  spec_powlaw_lmin_Ang = %.1f \n'%lmin
    phot_star_params = phot_star_params + '  spec_powlaw_lmax_Ang = %.1f \n'%lmax
    phot_star_params = phot_star_params + '# miscelaneous : \n'
    phot_star_params = phot_star_params + '  verbose              = T \n'
    phot_star_params = phot_star_params + '  nphot                = %i \n'%nphot
    phot_star_params = phot_star_params + '  ranseed              = %i \n'%seed
    phot_star_params = phot_star_params + '!--------------------------------------------------------------------------------\n \n'
    return phot_star_params



In [32]:
def get_cdd_params(ramDir,ts,domdump_repo,zoom,max_cells):
    cdd_params = '!-------------------------------------------------------------------------------- \n'
    cdd_params = cdd_params + '[CreateDomDump] \n'
    cdd_params = cdd_params + '# input outputs: \n'
    cdd_params = cdd_params + '  DomDumpDir         = %s \n'%domdump_repo
    cdd_params = cdd_params + '  repository         = %s \n'%ramDir
    cdd_params = cdd_params + '  snapnum            = %i \n'%ts
    cdd_params = cdd_params + '  reading_method     = hilbert_ion \n'
    cdd_params = cdd_params + '  ion_parameter_file = %s/params_ions.dat \n \n'%domdump_repo
    cdd_params = cdd_params + '# computational domain parameters \n'
    cdd_params = cdd_params + '  comput_dom_type    =  sphere \n'
    
    if(not zoom):
        print('Have to implement for the case when its not a zoom')
        return 0
    hcat = hC(ramDir,ts,HaloDir='Halos/',zoom=zoom)
    readFromRamsesOutput = True
    hcat.load_catalog()
    hmain = np.where((hcat.level==1) &(hcat.contam < 0.5) &(hcat.mvir > 4e7*1e-11))[0]
    myhalo = hmain[np.where(hcat.mvir[hmain]==max(hcat.mvir[hmain]))[0][0]]
    center = [hcat.x_cu[myhalo],hcat.y_cu[myhalo],hcat.z_cu[myhalo]]
    radius = hcat.rvir_cu[myhalo]
    
    cdd_params = cdd_params + '  comput_dom_pos     =  %.8f, %.8f, %.8f \n'%(center[0],center[1],center[2])
    cdd_params = cdd_params + '  comput_dom_rsp     =  %.5f \n \n'%radius
    cdd_params = cdd_params + '# domain decomposition parameters \n'
    cdd_params = cdd_params + '  decomp_dom_type    = sphere \n'
    cdd_params = cdd_params + '  decomp_dom_ndomain = 1 \n'
    cdd_params = cdd_params + '  decomp_dom_xc      = %.8f \n'%center[0]
    cdd_params = cdd_params + '  decomp_dom_yc      = %.8f \n'%center[1]
    cdd_params = cdd_params + '  decomp_dom_zc      = %.8f \n'%center[2]
    cdd_params = cdd_params + '  decomp_dom_rsp     = %.5f \n \n'%(radius*1.25)
    cdd_params = cdd_params + '# miscelaneous : \n'
    cdd_params = cdd_params + '  verbose            = T \n'
    cdd_params = cdd_params + '  max_cells          = %i \n'%max_cells
    cdd_params = cdd_params + '!--------------------------------------------------------------------------------\n \n'
    return cdd_params


In [27]:
def get_line_params(ion):
    line_params = '!-------------------------------------------------------------------------------- \n'
    line_params = line_params + '[lines] \n'
    if(ion=='CII'):
        line_params = line_params + 'ion_number   = 2 \n'
        line_params = line_params + 'm_ion        = 12.0164, 12.0164 \n'
        line_params = line_params + 'n_line       = 1, 2 \n'
        line_params = line_params + 'lambda       = 1.334532d3, 1.335663d3, 1.335708d3 \n'
        line_params = line_params + 'f            = 1.29d-1, 1.27d-2, 1.15d-1 \n'
        line_params = line_params + 'A            = 2.41d8, 4.76d7, 2.88d8 \n'
        line_params = line_params + 'n_fluo       = 1, 1, 0 \n'
        line_params = line_params + 'lambda_fluo  = 1.335663d3, 1.334532d3 \n'
        line_params = line_params + 'A_fluo       = 4.76d7, 2.41d8 \n'
        line_params = line_params + 'recoil       = F \n'
        line_params = line_params + 'isotropic    = T \n'
        line_params = line_params + '!--------------------------------------------------------------------------------\n \n'
        return line_params
    elif(ion=='SiII_1260'):
        line_params = line_params + 'ion_number   = 2 \n'
        line_params = line_params + 'm_ion        = 28.0855, 28.0855 \n'
        line_params = line_params + 'n_line       = 1, 2 \n'
        line_params = line_params + 'lambda       = 1260.42, 1264.73, 1265.02 \n'
        line_params = line_params + 'f            = 1.22d0, 1.09d0, 1.13d-1 \n'
        line_params = line_params + 'A            = 2.57d9, 4.04d9, 4.73d8 \n'
        line_params = line_params + 'n_fluo       = 1, 0, 1 \n'
        line_params = line_params + 'lambda_fluo  = 1265.02, 1260.42 \n'
        line_params = line_params + 'A_fluo       = 4.73d8, 2.57d9 \n'
        line_params = line_params + 'recoil       = F \n'
        line_params = line_params + 'isotropic    = T \n'
        line_params = line_params + '!--------------------------------------------------------------------------------\n \n'
    elif(ion=='SiII_1526'):
        line_params = line_params + 'ion_number   = 2 \n'
        line_params = line_params + 'm_ion        = 28.0855, 28.0855 \n'
        line_params = line_params + 'n_line       = 1, 1 \n'
        line_params = line_params + 'lambda       = 1526.72, 1533.45 \n'
        line_params = line_params + 'f            = 1.33d-1, 1.33d-1 \n'
        line_params = line_params + 'A            = 3.81d8, 7.52d8 \n'
        line_params = line_params + 'n_fluo       = 1, 1 \n'
        line_params = line_params + 'lambda_fluo  = 1533.45, 1526.72 \n'
        line_params = line_params + 'A_fluo       = 7.52d8, 3.81d8 \n'
        line_params = line_params + 'recoil       = F \n'
        line_params = line_params + 'isotropic    = T \n'
        line_params = line_params + '!--------------------------------------------------------------------------------\n \n'
    elif(ion=='OI_SiII'):
        line_params = line_params + 'ion_number   = 5 \n'
        line_params = line_params + 'm_ion        = 15.999, 15.999, 15.999, 28.0855, 28.0855 \n'
        line_params = line_params + 'n_line       = 1, 1, 1, 1, 1 \n'
        line_params = line_params + 'lambda       = 1302.168, 1304.858, 1306.029, 1304.37, 1309.27 \n'
        line_params = line_params + 'f            = 5.20d-2, 5.18d-2, 5.19d-2, 9.28d-2, 8d-2 \n'
        line_params = line_params + 'A            = 3.41d8, 2.03d8, 6.76d7, 3.64d8, 6.23d8 \n'
        line_params = line_params + 'n_fluo       = 2, 2, 2, 1, 1 \n'
        line_params = line_params + 'lambda_fluo  = 1304.858, 1306.029, 1302.168, 1306.029, 1302.168, 1304.858, 1309.27, 1304.37 \n'
        line_params = line_params + 'A_fluo       = 2.03d8, 6.76d7, 3.41d8, 6.76d7, 3.41d8, 2.03d8, 6.23d8, 3.64d8 \n'
        line_params = line_params + 'recoil       = F \n'
        line_params = line_params + 'isotropic    = T \n'
        line_params = line_params + '!--------------------------------------------------------------------------------\n \n'
    elif(ion=='AlII_1670'):
        line_params = line_params + 'ion_number   = 1 \n'
        line_params = line_params + 'm_ion        = 26.981539 \n'
        line_params = line_params + 'n_line       = 1 \n'
        line_params = line_params + 'lambda       = 1670.7867 \n'
        line_params = line_params + 'f            = 1.77d0 \n'
        line_params = line_params + 'A            = 1.41d9 \n'
        line_params = line_params + 'n_fluo       = 0 \n'
        line_params = line_params + 'lambda_fluo  = \n'
        line_params = line_params + 'A_fluo       = \n'
        line_params = line_params + 'recoil       = F \n'
        line_params = line_params + 'isotropic    = T \n'
        line_params = line_params + '!--------------------------------------------------------------------------------\n \n'
    elif(ion=='FeII_1608'):
        line_params = line_params + 'ion_number   = 2 \n'
        line_params = line_params + 'm_ion        = 55.845, 55.845 \n'
        line_params = line_params + 'n_line       = 1, 1 \n'
        line_params = line_params + 'lambda       = 1608.45078, 1618.46769 \n'
        line_params = line_params + 'f            = 5.91d-2, 2.17d-2 \n'
        line_params = line_params + 'A            = 1.91d8, 5.53d7 \n'
        line_params = line_params + 'n_fluo       = 1, 1 \n'
        line_params = line_params + 'lambda_fluo  = 1618.46769, 1608.45078 \n'
        line_params = line_params + 'A_fluo       = 5.53d7, 1.91d8 \n'
        line_params = line_params + 'recoil       = F \n'
        line_params = line_params + 'isotropic    = T \n'
        line_params = line_params + '!--------------------------------------------------------------------------------\n \n'
    elif(ion=='CIV_1548'):
        line_params = line_params + 'ion_number   = 1 \n'
        line_params = line_params + 'm_ion        = 12.0164 \n'
        line_params = line_params + 'n_line       = 2 \n'
        line_params = line_params + 'lambda       = 1548.202, 1550.774 \n'
        line_params = line_params + 'f            = 1.9d-1, 9.52d-2 \n'
        line_params = line_params + 'A            = 2.65d8, 2.64d8 \n'
        line_params = line_params + 'n_fluo       = 0, 0 \n'
        line_params = line_params + 'lambda_fluo  = \n'
        line_params = line_params + 'A_fluo       = \n'
        line_params = line_params + 'recoil       = F \n'
        line_params = line_params + 'isotropic    = T \n'
        line_params = line_params + '!--------------------------------------------------------------------------------\n \n'
    elif(ion=='MgII_2796'):
        line_params = line_params + 'ion_number   = 1 \n'
        line_params = line_params + 'm_ion        = 24.305 \n'
        line_params = line_params + 'n_line       = 2 \n'
        line_params = line_params + 'lambda       = 2796.352, 2803.530 \n'
        line_params = line_params + 'f            = 6.08d-1, 3.08d-1 \n'
        line_params = line_params + 'A            = 2.6d8, 2.57d8 \n'
        line_params = line_params + 'n_fluo       = 0, 0 \n'
        line_params = line_params + 'lambda_fluo  = \n'
        line_params = line_params + 'A_fluo       = \n'
        line_params = line_params + 'recoil       = F \n'
        line_params = line_params + 'isotropic    = T \n'
        line_params = line_params + '!--------------------------------------------------------------------------------\n \n'
    elif(ion=='SiIV_1393'):
        line_params = line_params + 'ion_number   = 1 \n'
        line_params = line_params + 'm_ion        = 28.0855 \n'
        line_params = line_params + 'n_line       = 2 \n'
        line_params = line_params + 'lambda       = 1393.76, 1402.77 \n'
        line_params = line_params + 'f            = 5.13d-1, 2.55d-1 \n'
        line_params = line_params + 'A            = 8.8d8, 8.63d8 \n'
        line_params = line_params + 'n_fluo       = 0, 0 \n'
        line_params = line_params + 'lambda_fluo  = \n'
        line_params = line_params + 'A_fluo       = \n'
        line_params = line_params + 'recoil       = F \n'
        line_params = line_params + 'isotropic    = T \n'
        line_params = line_params + '!--------------------------------------------------------------------------------\n \n'
    else:
        print('Line not implemented')
        return 0
        